In [4]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

##参数
f,cx,cy=1000,320,240

##加载2d点
file0= open("./data/image_formation0.xyz")
file1= open("./data/image_formation1.xyz")
points0=[]
points1=[]
for line in file0.readlines():
    if line=="\n":
        continue
    data=np.array(line.strip("\n").split(" ")).astype(np.float)
    points0.append(data)
points0=np.array(points0)[:,:2]
for line in file1.readlines():
    if line=="\n":
        continue
    data=np.array(line.strip("\n").split(" ")).astype(np.float)
    points1.append(data)
points1=np.array(points1)[:,:2]

In [5]:
##估计两视图相关姿势，即求F、E
F,mask=cv.findFundamentalMat(points0,points1)
K=np.mat([[f, 0, cx],[ 0, f, cy],[ 0, 0, 1]])
E=K.T*F*K
ret,R,t,mask=cv.recoverPose(E,points0,points1,K)

# ##重建3d点
P0=K*np.mat(np.eye(3,4))
Rt=np.concatenate((R,t.reshape(3,1)),axis=1)
P1=K*Rt
print(Rt)

[[ 0.95760331 -0.00869856 -0.28795874  0.71862356]
 [-0.06812347  0.96435967 -0.25567483  0.68308596]
 [ 0.2799198   0.26445181  0.92288144  0.13028335]]


In [6]:
# # t
X=cv.triangulatePoints(P0,P1,points0.T,points1.T)
X[0]=X[0]/X[3]
X[1]=X[1]/X[3]
X[2]=X[2]/X[3]
X[3]=1

# print(len(X[0]) )
for i in range(len(X[0])):
    with open("triangulation.xyz",'a') as f:
        f.write("{} {} {}\n".format(X[0,i],X[1,i],X[2,i]))

F:\A_Codes\C_PythonCodes\JupyteNotebook\CV